In [ ]:
%%html
<!-- P0 -->
<div class="aegis">
    <button type="button" class="jupyter-button jupyter-widgets mod-primary widget-button" onclick="initQB()">Initialise
        QB</button>
</div>
<script>
    $.ajaxSetup({
        headers: { "Authorization": localStorage.getItem("token") }
    });
    function initQB() {
        Jupyter.notebook.execute_cells([1]);
        Jupyter.notebook.execute_cells([2, 3, 4, 5, 6, 7, 8]);
    }

</script>

<style>
    .aegis .control-label {
        padding-right: 10px;
        font-weight: bold;
    }

    .aegis .widget-dropdown {
        width: 100%;
        height: inherit;
    }

    .aegis .widget-dropdown>select {
        margin-left: 0;
        margin-right: 0;
    }

    .aegis .input-group .jupyter-button {
        padding: 6px 12px;
        line-height: 1.42857143;
        /*height: 32px;*/
    }

    .aegis .form-control {
        padding-top: 0;
        padding-bottom: 0;
        height: 28px;
        border: 1px solid #9E9E9E;
        border-radius: 0;
        padding-left: 8px;
        -webkit-box-shadow: none;
        box-shadow: none;
        color: rgba(0, 0, 0, .8);
    }

    .aegis select[multiple],
    .aegis select[size] {
        height: auto;
    }

    .aegis #fileBrowser option[dir="true"] {
        font-weight: bold;
    }

    .aegis input[type=checkbox] {
        height: 28px;
    }

    .output pre {
        white-space: pre;
    }

    #list2 {
        width: 320px;
    }

    #list2 ol {
        font-size: 18px;
        color: #337ab7;
    }

    #list2 ol li p {
        padding: 8px;
        font-style: normal;
        font-size: 13px;
        color: black;
        border-left: 1px solid #999;
    }

    #list2 ol li.applied p em {
        color: lightgray;
    }

    #list2 ol li p em {
        display: block;
    }

    #filter-application-buttons {
        text-align: center;
    }

    .aegis h4 {
        margin: 9px 0px;
    }

    .aegis h4 span {
        font-style: italic;
        font-weight: initial;
    }
</style>


In [ ]:
#P1

from pyspark.sql.functions import col, isnan, isnull, quarter, ntile, month, hour, minute, dayofmonth, dayofyear, to_date, unix_timestamp, trunc, datediff, months_between, sqrt, when
from pyspark.sql.window import Window
from pyspark.sql import Row, functions as F
from pyspark.sql.functions import unix_timestamp
from  pyspark.sql.functions import abs as cabs
from pyspark.sql.functions import round as pyround
import json
import requests
from hops import hdfs
import pydoop.hdfs as phdfs
from  pyspark.sql.functions import year, ltrim, rtrim
from pyspark.sql.functions import coalesce, lit

myhdfs = phdfs.hdfs(user = hdfs.project_user())

tempDF = None
masterDF = None
hdfspath = hdfs.project_path()
g=None
query_init="from pyspark.sql.functions import col, isnan, isnull, quarter, ntile, month, hour, minute, dayofmonth, dayofyear, to_date, unix_timestamp, trunc, datediff, months_between, sqrt, when, unix_timestamp, year, ltrim, rtrim, coalesce, lit \nfrom pyspark.sql.window import Window \nfrom pyspark.sql import Row, functions as F\nfrom  pyspark.sql.functions import abs as cabs \nfrom pyspark.sql.functions import round as pyround\n"
query = query_init

def apply_filters(applied_filters):
    global tempDF
    global masterDF
    applied_filters = json.loads(applied_filters)
    global g
    global query
    g = applied_filters
    filters_success = []
    for i in range(1,len(applied_filters)+1):
        index = str(i)
        filter_type = applied_filters[index]["type"]
        if applied_filters[index]["executed"]:
            continue  
        try:
            if filter_type=="filter":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                if applied_filters[index]["parameters"][1]=="like":
                    tempDF = tempDF.filter(applied_filters[index]["parameters"][0]+" "+applied_filters[index]["parameters"][1]+" \'"+applied_filters[index]["parameters"][2][:-1]+"%\'")
                    query+="tempDF = tempDF.filter(\""+applied_filters[index]["parameters"][0]+" "+applied_filters[index]["parameters"][1]+" \'"+applied_filters[index]["parameters"][2][:-1]+"%\'\")\n"
                else:
                    try:
                        float(applied_filters[index]["parameters"][2])
                        tempDF = tempDF.filter(" ".join(applied_filters[index]["parameters"]))
                        query+="tempDF = tempDF.filter(\""+" ".join(applied_filters[index]["parameters"])+"\")\n"
                    except ValueError as ver:
                        applied_filters[index]["parameters"][2] = "'"+applied_filters[index]["parameters"][2]+"'"
                        tempDF = tempDF.filter(" ".join(applied_filters[index]["parameters"]))
                        query+="tempDF = tempDF.filter(\""+" ".join(applied_filters[index]["parameters"])+"\")\n"
            if filter_type=="filterOr":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                if applied_filters[index]["parameters"][1]=="like":
                    cond1 = applied_filters[index]["parameters"][0]+" "+applied_filters[index]["parameters"][1]+" \'"+applied_filters[index]["parameters"][2][:-1]+"%\'"
                else:
                    try:
                        float(applied_filters[index]["parameters"][2])
                    except ValueError as ver:       
                        applied_filters[index]["parameters"][2] = "'"+applied_filters[index]["parameters"][2]+"'"
                    cond1 = " ".join(applied_filters[index]["parameters"][:3])

                applied_filters[index]["parameters"][3] = "`"+applied_filters[index]["parameters"][3]+"`"
                if applied_filters[index]["parameters"][4]=="like":
                    cond2 = applied_filters[index]["parameters"][3]+" "+applied_filters[index]["parameters"][4]+" \'"+applied_filters[index]["parameters"][5][:-1]+"%\'"
                else:
                    try:
                        float(applied_filters[index]["parameters"][5])
                    except ValueError as ver:       
                        applied_filters[index]["parameters"][5] = "'"+applied_filters[index]["parameters"][5]+"'"
                    cond2 = " ".join(applied_filters[index]["parameters"][3:6])

                tempDF = tempDF.filter(cond1 + " or " + cond2)
                query+="tempDF = tempDF.filter(\""+cond1 + " or " + cond2+"\")\n"
            elif filter_type=="join":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                keep = [x for x in paramsList[1:] if (None not in x)]
                jhow = str(paramsList[0])
                if len(keep) == 1:
                    tempDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1])],how=jhow)
                elif len(keep) == 2:
                    tempDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1]),col('masterDF.'+keep[1][0])==col('tempDF.'+keep[1][1])],how=jhow)
                elif len(keep) == 3:
                    tempDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1]),col('masterDF.'+keep[1][0])==col('tempDF.'+keep[1][1]),col('masterDF.'+keep[2][0])==col('tempDF.'+keep[2][1])],how=jhow)
            elif filter_type=="year":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],year(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",year(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="quarter":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],quarter(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",quarter(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="month":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],month(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",month(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="hour":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],hour(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",hour(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="minute":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],minute(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",minute(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="dayofmonth":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],dayofmonth(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",dayofmonth(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="dayofyear":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],dayofyear(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",dayofyear(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="to_date":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],to_date(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",to_date(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="unix_timestamp":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],unix_timestamp(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",to_timestamp(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="ytrunc":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],trunc(tempDF[applied_filters[index]["parameters"][0]],"year"))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",trunc(tempDF[\""+applied_filters[index]["parameters"][0]+"\"],\"year\"))\n"
            elif filter_type=="mtrunc":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],trunc(tempDF[applied_filters[index]["parameters"][0]],"month"))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",trunc(tempDF[\""+applied_filters[index]["parameters"][0]+"\"],\"month\"))\n"
            elif filter_type=="sqrt":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = applied_filters[index]["parameters"][1].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][1],sqrt(tempDF[applied_filters[index]["parameters"][0]]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][1]+"\",sqrt(tempDF[\""+applied_filters[index]["parameters"][0]+"\"]))\n"
            elif filter_type=="round":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = int(applied_filters[index]["parameters"][1])
                applied_filters[index]["parameters"][2] = applied_filters[index]["parameters"][2].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][2],pyround(tempDF[applied_filters[index]["parameters"][0]],applied_filters[index]["parameters"][1]))
                query += "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][2]+"\",pyround(tempDF[\""+applied_filters[index]["parameters"][0]+"\"],"+str(applied_filters[index]["parameters"][1])+"))\n"
            elif filter_type=="when_1":
                then_value = None
                else_value = None
                new_col_name = applied_filters[index]["parameters"][5].replace(" ","_")
                applied_filters[index]["parameters"][0] = "tempDF['`"+applied_filters[index]["parameters"][0]+"`']"
                applied_filters[index]["parameters"][2] = "'"+applied_filters[index]["parameters"][2]+"'"
                condition = " ".join(applied_filters[index]["parameters"][0:3])
                evaluated_condition = eval(condition)
                try:
                    then_value = float(applied_filters[index]["parameters"][3])
                except ValueError as ver:
                    then_value = "'"+applied_filters[index]["parameters"][3]+"'"
                try:
                    else_value = float(applied_filters[index]["parameters"][4])
                except ValueError as ver:
                    else_value = "'"+applied_filters[index]["parameters"][4]+"'"
                tempDF = tempDF.withColumn(new_col_name,when(evaluated_condition,then_value).otherwise(else_value))
                query+="tempDF = tempDF.withColumn(\""+new_col_name+"\",when("+condition+",\""+str(then_value)+"\").otherwise(\""+str(else_value)+"\"))\n"
            elif filter_type=="when_2":
                new_col_name = applied_filters[index]["parameters"][9].replace(" ","_")
                applied_filters[index]["parameters"][0] = "tempDF['`"+applied_filters[index]["parameters"][0]+"`']"
                applied_filters[index]["parameters"][2] = "'"+applied_filters[index]["parameters"][2]+"'"
                condition = " ".join(applied_filters[index]["parameters"][0:3])
                evaluated_condition = eval(condition)
                applied_filters[index]["parameters"][3] = "tempDF['`"+applied_filters[index]["parameters"][3]+"`']"
                applied_filters[index]["parameters"][5] = "'"+applied_filters[index]["parameters"][5]+"'"
                then_condition = " ".join(applied_filters[index]["parameters"][3:6])
                evaluated_then_condition = eval(then_condition)
                applied_filters[index]["parameters"][6] = "tempDF['`"+applied_filters[index]["parameters"][6]+"`']"
                applied_filters[index]["parameters"][8] = "'"+applied_filters[index]["parameters"][8]+"'"
                else_condition = " ".join(applied_filters[index]["parameters"][6:9])
                evaluated_else_condition = eval(else_condition)
                tempDF = tempDF.withColumn(new_col_name,when(evaluated_condition,evaluated_then_condition).otherwise(evaluated_else_condition))
                query+= "tempDF = tempDF.withColumn(\""+new_col_name+"\",when("+condition+","+then_condition+").otherwise("+else_condition+"))\n"
            elif filter_type=="when_3":
                new_col_name = applied_filters[index]["parameters"][9].replace(" ","_")
                applied_filters[index]["parameters"][0] = "tempDF['`"+applied_filters[index]["parameters"][0]+"`']"
                applied_filters[index]["parameters"][2] = "'"+applied_filters[index]["parameters"][2]+"'"
                condition = " ".join(applied_filters[index]["parameters"][0:3])
                evaluated_condition = eval(condition)
                applied_filters[index]["parameters"][3] = "tempDF['`"+applied_filters[index]["parameters"][3]+"`']"
                applied_filters[index]["parameters"][5] = "tempDF['`"+applied_filters[index]["parameters"][5]+"`']"
                then_condition = " ".join(applied_filters[index]["parameters"][3:6])
                evaluated_then_condition = eval(then_condition)
                applied_filters[index]["parameters"][6] = "tempDF['`"+applied_filters[index]["parameters"][6]+"`']"
                applied_filters[index]["parameters"][8] = "tempDF['`"+applied_filters[index]["parameters"][8]+"`']"
                else_condition = " ".join(applied_filters[index]["parameters"][6:9])
                evaluated_else_condition = eval(else_condition)
                tempDF = tempDF.withColumn(new_col_name,when(evaluated_condition,evaluated_then_condition).otherwise(evaluated_else_condition))
                query+= "tempDF = tempDF.withColumn(\""+new_col_name+"\",when("+condition+","+then_condition+").otherwise("+else_condition+"))\n"
            elif filter_type=="col_operation_1":
                new_col_name = applied_filters[index]["parameters"][3].replace(" ","_")
                applied_filters[index]["parameters"][0] = "tempDF['`"+applied_filters[index]["parameters"][0]+"`']"
                applied_filters[index]["parameters"][2] = "'"+applied_filters[index]["parameters"][2]+"'"
                expr = " ".join(applied_filters[index]["parameters"][0:3])
                evaluated_expr = eval(expr)
                tempDF = tempDF.withColumn(new_col_name,evaluated_expr)
                query+="tempDF = tempDF.withColumn(\""+new_col_name+"\","+expr+")\n"
            elif filter_type=="col_operation_2":
                new_col_name = applied_filters[index]["parameters"][3].replace(" ","_")
                applied_filters[index]["parameters"][0] = "tempDF['`"+applied_filters[index]["parameters"][0]+"`']"
                applied_filters[index]["parameters"][2] = "tempDF['`"+applied_filters[index]["parameters"][2]+"`']"
                expr = " ".join(applied_filters[index]["parameters"][0:3])
                evaluated_expr = eval(expr)
                tempDF = tempDF.withColumn(new_col_name,evaluated_expr)
                query+="tempDF = tempDF.withColumn(\""+new_col_name+"\","+expr+")\n"
            elif filter_type=="datediff":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = "`"+applied_filters[index]["parameters"][1]+"`"
                applied_filters[index]["parameters"][2] = applied_filters[index]["parameters"][2].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][2],datediff(tempDF[applied_filters[index]["parameters"][0]],tempDF[applied_filters[index]["parameters"][1]]))
                query+="tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][2]+"\",datediff(tempDF[\""+applied_filters[index]["parameters"][0]+"\"],tempDF[\""+applied_filters[index]["parameters"][1]+"\"]))\n"
            elif filter_type=="months_between":
                applied_filters[index]["parameters"][0] = "`"+applied_filters[index]["parameters"][0]+"`"
                applied_filters[index]["parameters"][1] = "`"+applied_filters[index]["parameters"][1]+"`"
                applied_filters[index]["parameters"][2] = applied_filters[index]["parameters"][2].replace(" ","_")
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][2],months_between(tempDF[applied_filters[index]["parameters"][0]],tempDF[applied_filters[index]["parameters"][1]]))
                query+="tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][2]+"\",months_between(tempDF[\""+applied_filters[index]["parameters"][0]+"\"],tempDF[\""+applied_filters[index]["parameters"][1]+"\"]))\n"
            elif filter_type=="orderBy":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                l = ["`"+x+"`" for x in list(paramsList[0])]
                tempDF = tempDF.orderBy(l,ascending = list(paramsList[1]))
                query+="tempDF = tempDF.orderBy(\""+l[0]+"\",ascending = "+str(list(paramsList[1]))+")\n"
            elif filter_type=="drop":
                tempDF = tempDF.drop(*applied_filters[index]["parameters"][0])
                query+="tempDF = tempDF.drop(*"+str(applied_filters[index]["parameters"][0])+")\n"
            elif filter_type=="withColumnRenamed":
                tempDF = tempDF.withColumnRenamed(applied_filters[index]["parameters"][0],applied_filters[index]["parameters"][1])
                query+="tempDF = tempDF.withColumnRenamed(\""+applied_filters[index]["parameters"][0]+"\",\""+applied_filters[index]["parameters"][1]+"\")\n"
            elif filter_type=="select":
                l = ["`"+x+"`" for x in applied_filters[index]["parameters"][0]]
                tempDF = tempDF.select(*l)
                query+="tempDF = tempDF.select(*"+str(l)+")\n"
            elif filter_type=="dropDuplicates":
                tempDF = tempDF.dropDuplicates()
                query+="tempDF = tempDF.dropDuplicates()\n"
            elif filter_type=="fillna":
                fillDict = dict(applied_filters[index]["parameters"][0])
                tempDF = tempDF.fillna(fillDict)
                query+="tempDF = tempDF.fillna("+str(fillDict)+")\n"
            elif filter_type=="replace":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                tempDF = tempDF.replace(paramsList[0],paramsList[1],str(paramsList[2]))
                query+="tempDF = tempDF.replace(list("+str(paramsList[0])+"),list("+str(paramsList[1])+"),(\""+str(paramsList[2])+"\"))\n"
            elif filter_type=="min":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                colsp = list(paramsList[1])
                cols = ["`"+x+"`" for x in colsp]
                gby = ["`"+y+"`" for y in list(paramsList[0])]
                tempDF = tempDF.groupBy(gby).min(*cols)
                query+="tempDF = tempDF.groupBy(\""+gby[0]+"\").min(*"+str(cols)+")\n"
            elif filter_type=="max":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                colsp = list(paramsList[1])
                cols = ["`"+x+"`" for x in colsp]
                gby = ["`"+y+"`" for y in list(paramsList[0])]
                tempDF = tempDF.groupBy(gby).max(*cols)
                query+="tempDF = tempDF.groupBy(\""+gby[0]+"\").max(*"+str(cols)+")\n"
            elif filter_type=="avg":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                colsp = list(paramsList[1])
                cols = ["`"+x+"`" for x in colsp]
                gby = ["`"+y+"`" for y in list(paramsList[0])]
                tempDF = tempDF.groupBy(gby).avg(*cols)
                query+="tempDF = tempDF.groupBy(\""+gby[0]+"\").avg(*"+str(cols)+")\n"
            elif filter_type=="sum":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                colsp = list(paramsList[1])
                cols = ["`"+x+"`" for x in colsp]
                gby = ["`"+y+"`" for y in list(paramsList[0])]
                tempDF = tempDF.groupBy(gby).sum(*cols)
                query+="tempDF = tempDF.groupBy(\""+gby[0]+"\").sum(*"+str(cols)+")\n"
            elif filter_type=="count":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                l = ["`"+x+"`" for x in paramsList[0]]
                tempDF = tempDF.groupBy(list(l)).count()
                query+="tempDF.groupBy(\""+list(l)[0]+"\").count()\n"
            elif filter_type=="isnan":
                cols = "`"+applied_filters[index]["parameters"][0]+"`"
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][0]+"_isnan",isnan(tempDF[cols]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][0]+"_isnan\",isnan(tempDF[\""+cols+"\"]))\n"
            elif filter_type=="isnull":
                cols = "`"+applied_filters[index]["parameters"][0]+"`"
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][0]+"_isnull",isnull(tempDF[cols]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][0]+"_isnull\",isnull(tempDF[\""+cols+"\"]))\n"
            elif filter_type=="ltrim":
                cols = "`"+applied_filters[index]["parameters"][0]+"`"
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][0]+"_ltrimmed",ltrim(tempDF[cols]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][0]+"_ltrimmed\",ltrim(tempDF[\""+cols+"\"]))\n"
            elif filter_type=="rtrim":
                cols = "`"+applied_filters[index]["parameters"][0]+"`"
                tempDF = tempDF.withColumn(applied_filters[index]["parameters"][0]+"_rtrimmed",rtrim(tempDF[cols]))
                query+= "tempDF = tempDF.withColumn(\""+applied_filters[index]["parameters"][0]+"_rtrimmed\",rtrim(tempDF[\""+cols+"\"]))\n"
            elif filter_type=="joinApproxTime":
                paramsList = [x for x in applied_filters[index]["parameters"]]
                keep = [x for x in paramsList[1:-1] if (None not in x)]
                jhow = str(paramsList[0])
                time_columns = paramsList[-1]
                allcols = tempDF.columns
                allcols.extend(masterDF.columns)
                if len(keep) == 1:
                    joinedDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1])],how=jhow)
                    additional_col = joinedDF.withColumn("time_diff",  cabs(unix_timestamp(str(time_columns[0])) - unix_timestamp(str(time_columns[1]))))
                    partDF = additional_col.select(F.row_number().over(Window.partitionBy(str(keep[0][0])).orderBy("time_diff")).alias("rank"), "time_diff",*allcols ).filter(col("rank") == 1)
                    tempDF = partDF
                elif len(keep) == 2:
                    joinedDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1]),col('masterDF.'+keep[1][0])==col('tempDF.'+keep[1][1])],how=jhow)
                    additional_col = joinedDF.withColumn("time_diff",  cabs(unix_timestamp(str(time_columns[0])) - unix_timestamp(str(time_columns[1]))))
                    partDf = additional_col.select( F.row_number().over(Window.partitionBy(str(keep[0][0]),str(keep[1][0])).orderBy("time_diff")).alias("rank"), *allcols ).filter(col("rank") == 1).drop('rank')
                    tempDF = partDF
                elif len(keep) == 3:
                    joinedDF = masterDF.alias('masterDF').join(tempDF.alias('tempDF'),on=[col('masterDF.'+keep[0][0])==col('tempDF.'+keep[0][1]),col('masterDF.'+keep[1][0])==col('tempDF.'+keep[1][1]),col('masterDF.'+keep[2][0])==col('tempDF.'+keep[2][1])],how=jhow)
                    additional_col = joinedDF.withColumn("time_diff",  cabs(unix_timestamp(str(time_columns[0])) - unix_timestamp(str(time_columns[1]))))
                    partDf = additional_col.select( F.row_number().over(Window.partitionBy(str(keep[0][0]),str(keep[1][0]),str(keep[2][0])).orderBy("time_diff")).alias("rank"), *allcols).filter(col("rank") == 1).drop('rank')
                    tempDF = partDF
            filters_success.append(index)
        except Exception as e:
            g = e
            return
    return filters_success

def read_dataset(selectedFile, separator):
    global tempDF
    global query
    global g
    if selectedFile.startswith('/'):
        selectedFile = "hdfs://" + myhdfs.host + ":" + str(myhdfs.port) + selectedFile
    try:
        tempDF = spark.read.csv(selectedFile, header=True, inferSchema=True, sep=separator)
    except Exception as e:
        g=e
    query=query_init
    return tempDF.columns

def getTempColumns():
    global tempDF
    return tempDF.columns

def tempDFtoMasterDF():
    global masterDF
    global tempDF
    masterDF = tempDF
    return masterDF.columns

def saveMasterAsCSV(savepath, withheader):
    global masterDF
    global hdfspath
    global g
    hdfs_start_path = "hdfs://"+myhdfs.host+":"+str(myhdfs.port)
    completepath = hdfs_start_path + savepath
    if withheader == 1:
        headerflag = True
    else:
        headerflag = False
    try:
        masterDF.coalesce(1).write.csv(path=completepath, mode='append', header=headerflag)
        return 200
    except Exception as e:
        g=e

def getCSVfiles():
    fileSet = set()
    for f in myhdfs.walk(hdfs.project_path()):
        fileSet.add(f['name'])
    available_files = [x for x in list(fileSet) if x.endswith(".csv")]
    return available_files
    
def find_current_projectID(pDict):
    projectsDict = json.loads(pDict)
    pname = hdfs.project_name()
    return projectsDict[pname] #KeyError should not be possible here, so no try-except...
    
def walk_dataset(dataset_path):
    fileSet = set()
    hdfs_start_path = "hdfs://"+myhdfs.host+":"+str(myhdfs.port)
    for f in myhdfs.walk(hdfs_start_path + dataset_path):
        fileSet.add(f['name'])
    available_files = [x for x in list(fileSet) if x.endswith(".csv")]
    return available_files


In [ ]:
%%html
<!-- P2 -->
<div class="aegis">
    <div id="cbrowser">
        <div class="form-group row form-horizontal">
            <label class="col-sm-2 control-label">File:</label>
            <div class="col-sm-10">
                <div class="input-group widget-dropdown">
                    <select id="fileBrowser" class="form-control">
                    </select>
                    <span class="input-group-btn">
                        <button type="button" style="display:none" class="btn jupyter-button mod-primary" onclick="fileBrowserOpen()">
                            Open
                        </button>
                        <button type="button" class="btn jupyter-button mod-primary" onclick="forceFileBrowserRefresh()">
                            Refresh list
                        </button>
                    </span>
                </div>
            </div>
        </div>
        <div class="form-group row form-horizontal" style="display:none">
            <label class="col-sm-2 control-label">Available Datasets:</label>
            <div class="col-sm-10">
                <div class="input-group widget-dropdown">
                    <select id="datasetSelect" class="form-control">
                    </select>
                    <span class="input-group-btn">
                        <button type="button" class="btn jupyter-button mod-primary" onclick="selectDataset()"> Open
                            dataset
                        </button>
                        <button type="button" class="btn jupyter-button mod-primary" onclick="showCurrentBrowser()">
                            Refresh list
                        </button>
                    </span>
                </div>
            </div>
        </div>
        <div class="form-group row form-horizontal" style="display:none">
            <label class="col-sm-2 control-label">Available Files:</label>
            <div class="col-sm-10">
                <div class="widget-dropdown">
                    <select id="fileSelect" class="form-control">
                    </select>
                </div>
            </div>
        </div>
        <div class="form-group row form-horizontal">
            <label class="col-sm-2 control-label">CSV Separator:</label>
            <div class="col-sm-4">
                <div class="widget-dropdown">
                    <select id="sepSelect" class="form-control">
                        <option value=",">Comma (,)</option>
                        <option value=";">Semicolon (;)</option>
                    </select>
                </div>
            </div>
            <label class="col-sm-2 control-label">Encoding:</label>
            <div class="col-sm-4">
                <div class="widget-dropdown">
                    <select id="encSelect" class="form-control">
                        <option data-value="auto" value="auto">Auto</option>
                        <option data-value="ascii" value="ascii">ASCII</option>
                        <option data-value="utf_8" value="utf_8">UTF-8</option>
                        <option data-value="utf_8_sig" value="utf_8_sig">UTF-8 with BOM</option>
                        <option data-value="cp1252" value="cp1252">Windows-1252</option>
                    </select>
                </div>
            </div>
        </div>
        <div class="form-group row form-horizontal">
            <div class="col-sm-offset-2 col-sm-10">
                <button type="button" class="jupyter-button jupyter-widgets mod-primary widget-button" style="display:none"
                    onclick="selectDatafile()"> Open file </button>
                <button id="fileBrowserOpenBtn" type="button" class="btn jupyter-button mod-primary" onclick="fileBrowserOpen()">
                    Open
                </button>
            </div>
        </div>
    </div>
    <hr>

    <div id="filters">
        <div class="form-group row form-horizontal" id="null-filterType" style="display:none;">
            <label class="col-sm-2 control-label">Available actions:</label>
            <div class="col-sm-10">
                <div class="widget-dropdown">
                    <select id="filterType" class="form-control" id="filterType" onchange="divVisibilityChanged()">
                        <option value=""></option>
                        <option disabled>-------- Basic --------</option>
                        <option value="filter">Filter</option>
                        <option value="filterOr">Filter OR</option>
                        <option value="orderBy">Order By</option>
                        <option value="select">Select column(s)</option>
                        <option value="drop">Drop column(s)</option>
                        <option value="withColumnRenamed">Rename Column</option>
                        <option disabled>------ Aggregations ------</option>
                        <option value="count">Count</option>
                        <option value="min">Min</option>
                        <option value="max">Max</option>
                        <option value="avg">Average</option>
                        <option value="sum">Sum</option>
                        <option disabled>------- Cleansing -------</option>
                        <option value="fillna">Fill N/A with value</option>
                        <option value="isnan">Check if NaN</option>
                        <option value="isnull">Check if null</option>
                        <option value="replace">Replace value</option>
                        <option value="ltrim">Left Trim a Column</option>
                        <option value="rtrim">Right Trim a Column</option>
                        <option disabled value="linear_interp">Fill N/A with linear interpolation</option>
                        <option disabled>--------- Joins ---------</option>
                        <option value="join">Join with master</option>
                        <option value="joinApproxTime">Join with master on approximate time</option>
                        <option disabled>------- Time Conversions ------</option>
                        <option value="year">Add Year Colum</option>
                        <option value="quarter">Add Quarter Column</option>
                        <option value="month">Add Month Column</option>
                        <option value="hour">Add Hour Column</option>
                        <option value="minute">Add Minute Column</option>
                        <option value="to_date">Convert to date</option>
                        <option value="unix_timestamp">Convert to UNIX timestamp</option>
                        <option value="ytrunc">Truncate date to year</option>
                        <option value="mtrunc">Truncate date to month</option>
                        <option value="dayofyear">Add dayOfYear column</option>
                        <option value="dayofmonth">Add dayOfMonth column</option>
                        <option value="datediff">Number of days between dates</option>
                        <option value="months_between">Number of months between dates</option>
                        <option disabled>--------- Math ---------</option>
                        <option value="sqrt">Add Square Root Column</option>
                        <option value="round">Add Round Column</option>
                        <option disabled>--------- Other ---------</option>
                        <option value="col_operation_1">Add column - based on other column </option>
                        <option value="col_operation_2">Add column - combine two columns </option>
                        <option value="when_1">Add column - column-based condition</option>
                        <option value="when_2">Add column - column-based condition & value</option>
                        <option value="when_3">Add column - column-based condition & columns-combination value</option>
                        <option disabled value="replace_extract">Replace based on regexp</option>
                    </select>
                </div>
            </div>
        </div>

        <div id="filterType_filter" style="display:none">
            <h3>Filter</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector">
                            <option></option>
                            <option value="<">&lt;</option>
                            <option value="==">=</option>
                            <option value=">">&gt;</option>
                            <option value="!=">!=</option>
                            <option value="like">starts with</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div id="filterType_filterOr" style="display:none">
                <h3>Filter OR</h3>
                <div class="form-group row form-horizontal">
                    <label class="col-sm-2 control-label">Column 1:</label>
                    <div class="col-sm-10">
                        <div class="widget-dropdown">
                            <select class="form-control single-column-selector temp-column col1_colname">
                            </select>
                        </div>
                    </div>
                </div>
                <div class="form-group row form-horizontal">
                    <label class="col-sm-2 control-label">Operator:</label>
                    <div class="col-sm-10">
                        <div class="widget-dropdown">
                            <select class="form-control operator-selector col1_operator">
                                <option></option>
                                <option value="<">&lt;</option>
                                <option value="==">=</option>
                                <option value=">">&gt;</option>
                                <option value="!=">!=</option>
                                <option value="like">starts with</option>
                            </select>
                        </div>
                    </div>
                </div>
                <div class="form-group row form-horizontal">
                    <label class="col-sm-2 control-label">Value:</label>
                    <div class="col-sm-10">
                        <input class="form-control user-defined-value col1_value" />
                    </div>
                </div>
                <div class="form-group row form-horizontal">
                        <label class="col-sm-2 control-label">Column 2:</label>
                        <div class="col-sm-10">
                            <div class="widget-dropdown">
                                <select class="form-control single-column-selector temp-column col2_colname">
                                </select>
                            </div>
                        </div>
                    </div>
                    <div class="form-group row form-horizontal">
                        <label class="col-sm-2 control-label">Operator:</label>
                        <div class="col-sm-10">
                            <div class="widget-dropdown">
                                <select class="form-control operator-selector col2_operator">
                                    <option></option>
                                    <option value="<">&lt;</option>
                                    <option value="==">=</option>
                                    <option value=">">&gt;</option>
                                    <option value="!=">!=</option>
                                    <option value="like">starts with</option>
                                </select>
                            </div>
                        </div>
                    </div>
                    <div class="form-group row form-horizontal">
                        <label class="col-sm-2 control-label">Value:</label>
                        <div class="col-sm-10">
                            <input class="form-control user-defined-value col2_value" />
                        </div>
                    </div>
            </div>

        <div id="filterType_col_operation_1" style="display:none">
            <h3>Create column from an existing</h3>
            <h4>Compute new value <span>-only numeric operations supported</span></h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector">
                            <option></option>
                            <option value="+">+</option>
                            <option value="-">-</option>
                            <option value="*">*</option>
                            <option value="/">/</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value operation-value" />
                </div>
            </div>
            <h4>New Column Name</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value new-col-name" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_col_operation_2" style="display:none">
            <h3>Create column from two existing columns</h3>
            <h4>Compute new value <span>-only numeric operations supported</span></h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <select class="form-control single-column-selector temp-column temp-column-A">
                    </select>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector">
                            <option></option>
                            <option value="+">+</option>
                            <option value="-">-</option>
                            <option value="*">*</option>
                            <option value="/">/</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <select class="form-control single-column-selector temp-column temp-column-B">
                    </select>
                </div>
            </div>
            <h4>New Column Name</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value new-col-name" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_when_1" style="display:none">
            <h3>When-Then-Else</h3>
            <h4>Condition (When)</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column when-div">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector when-div">
                            <option></option>
                            <option value="<">&lt;</option>
                            <option value="==">=</option>
                            <option value=">">&gt;</option>
                            <option value="!=">!=</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value when-div" />
                </div>
            </div>
            <h4>Value (Then)</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value then-div" />
                </div>
            </div>
            <h4>Alternative Value (Else)</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Alt Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value else-div" />
                </div>
            </div>
            <h4>New Column Name</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value new-col-name" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_when_2" style="display:none">
            <h3>When-Then-Else</h3>
            <h4>Condition (When)</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column when-div temp-column-A">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector when-div">
                            <option></option>
                            <option value="<">&lt;</option>
                            <option value="==">=</option>
                            <option value=">">&gt;</option>
                            <option value="!=">!=</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value when-div" />
                </div>
            </div>
            <h4>Value (Then) <span>-only numeric operations supported</span></h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column then-div temp-column-A">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector then-div">
                            <option></option>
                            <option value="+">+</option>
                            <option value="-">-</option>
                            <option value="*">*</option>
                            <option value="/">/</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value then-div" />
                </div>
            </div>
            <h4>Alternative Value (Else) <span>-only numeric operations supported</span></h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column else-div temp-column-A">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector else-div">
                            <option></option>
                            <option value="+">+</option>
                            <option value="-">-</option>
                            <option value="*">*</option>
                            <option value="/">/</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Alt Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value else-div" />
                </div>
            </div>
            <h4>New Column Name</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value new-col-name" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_when_3" style="display:none">
            <h3>When-Then-Else</h3>
            <h4>Condition (When)</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column when-div">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector when-div">
                            <option></option>
                            <option value="<">&lt;</option>
                            <option value="==">=</option>
                            <option value=">">&gt;</option>
                            <option value="!=">!=</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value when-div" />
                </div>
            </div>
            <h4>Value (Then) <span>-only numeric operations supported</span></h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column then-div temp-column-A">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector then-div">
                            <option></option>
                            <option value="+">+</option>
                            <option value="-">-</option>
                            <option value="*">*</option>
                            <option value="/">/</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column then-div temp-column-B">
                        </select>
                    </div>
                </div>
            </div>
            <h4>Alternative Value (Else) <span>-only numeric operations supported</span></h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column else-div temp-column-A">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Operator:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector else-div">
                            <option></option>
                            <option value="+">+</option>
                            <option value="-">-</option>
                            <option value="*">*</option>
                            <option value="/">/</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column else-div temp-column-B">
                        </select>
                    </div>
                </div>
            </div>
            <h4>New Column Name</h4>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value new-col-name" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_orderBy" style="display:none">
            <h3>Order By</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Sort:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector">
                            <option value="1">Asc</option>
                            <option value="0">Desc</option>
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_count" style="display:none">
            <h3>Count</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Group By:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_ltrim" style="display:none">
            <h3>Left Trim</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_rtrim" style="display:none">
            <h3>Right Trim</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_isnan" style="display:none">
            <h3>Is NaN</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_isnull" style="display:none">
            <h3>Is Null</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_min" style="display:none">
            <h3>Min</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Columns:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select multiple class="form-control multiple-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Group By:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>

        </div>

        <div class="form-horizontal" id="filterType_max" style="display:none">
            <h3>Max</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Columns:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select multiple class="form-control multiple-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Group By:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_avg" style="display:none">
            <h3>Average</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Columns:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select multiple class="form-control multiple-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Group By:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_sum" style="display:none">
            <h3>Sum</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Columns:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select multiple class="form-control multiple-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Group By:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_year" style="display:none">
            <h3>Create Year Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_quarter" style="display:none">
            <h3>Create Quarter Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_month" style="display:none">
            <h3>Create Month Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_hour" style="display:none">
            <h3>Create Hour Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_minute" style="display:none">
            <h3>Create Minute Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_dayofmonth" style="display:none">
            <h3>Create Day of Month Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_dayofyear" style="display:none">
            <h3>Create Day of Year Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_to_date" style="display:none">
            <h3>Create Date Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_unix_timestamp" style="display:none">
            <h3>Create UNIX Timestamp Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_ytrunc" style="display:none">
            <h3>Truncate Date Column to Year</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_mtrunc" style="display:none">
            <h3>Truncate Date Column to Month</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_datediff" style="display:none">
            <h3>Create DateDiff Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column A:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column temp-column-A">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column B:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column temp-column-B">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_months_between" style="display:none">
            <h3>Create MonthsDiff Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column A:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column temp-column-A">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Date Column B:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column temp-column-B">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_sqrt" style="display:none">
            <h3>Create SQRT Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Select Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_round" style="display:none">
            <h3>Create Rounded Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Select Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Name of New Column:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value col-name" />
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Number of Decimal Places:</label>
                <div class="col-sm-10">
                    <input type="number" step="1" class="form-control user-defined-value int-input" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_select" style="display:none">
            <h3>Select Columns</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Columns:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select multiple class="form-control multiple-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_drop" style="display:none">
            <h3>Select Columns</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Columns:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select multiple class="form-control multiple-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_fillna" style="display:none">
            <h3>Fill N/A</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_replace" style="display:none">
            <h3>Replace Value</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Current Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value existing-value" />
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">To-Be Value:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value to-be-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_withColumnRenamed" style="display:none">
            <h3>Rename Column</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-column">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">New Name:</label>
                <div class="col-sm-10">
                    <input class="form-control user-defined-value" />
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_join" style="display:none">
            <h3>Join</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Join Type:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector">
                            <option value="inner">inner</option>
                            <option value="outer">outer</option>
                            <option value="left_outer">left_outer</option>
                            <option value="right_outer">right_outer</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Master Column 1:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector master-column master-col-1">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Temp Column 1:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-col-1 temp-column empty-option-allowed">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Master Column 2 (opt) :</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector master-column master-col-2">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Temp Column 2 (opt) :</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-col-2 temp-column empty-option-allowed">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Master Column 3 (opt) :</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector master-column master-col-3">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Temp Column 3 (opt) :</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-col-3 temp-column empty-option-allowed">
                        </select>
                    </div>
                </div>
            </div>
        </div>

        <div class="form-horizontal" id="filterType_joinApproxTime" style="display:none">
            <h3>Join on closest time</h3>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Join Type:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control operator-selector">
                            <option value="inner">inner</option>
                            <option value="outer">outer</option>
                            <option value="left_outer">left_outer</option>
                            <option value="right_outer">right_outer</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Master Time Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector master-column master-time-col">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Temp Time Column:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-time-col temp-column empty-option-allowed">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Master Column 1:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector master-column master-col-1">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Temp Column 1:</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-col-1 temp-column empty-option-allowed">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Master Column 2 (opt) :</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector master-column master-col-2">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Temp Column 2 (opt) :</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-col-2 temp-column empty-option-allowed">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Master Column 3 (opt) :</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector master-column master-col-3">
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group row form-horizontal">
                <label class="col-sm-2 control-label">Temp Column 3 (opt) :</label>
                <div class="col-sm-10">
                    <div class="widget-dropdown">
                        <select class="form-control single-column-selector temp-col-3 temp-column empty-option-allowed">
                        </select>
                    </div>
                </div>
            </div>
        </div>
    </div>
    <div id="filter-control-buttons" style="display:none">
        <div class="form-group row form-horizontal">
            <div class="col-sm-offset-2 col-sm-10">
                <button class="jupyter-button jupyter-widgets mod-primary widget-button" onclick="addFilter();"> OK
                </button>
                <button class="jupyter-button jupyter-widgets widget-button" onclick="cancelFilter()"> Cancel </button>
            </div>
        </div>
    </div>

    <div id="list2">
        <ol id="show-filters"></ol>
    </div>

    <div id="filter-application-buttons" style="display:none">
        <hr>
        <div class="form-group row form-horizontal">
            <button type="button" class="jupyter-button jupyter-widgets mod-primary widget-button"
                onclick="applyAllFilters();"> Apply Filters </button>
            <button type="button" class="jupyter-button jupyter-widgets mod-primary widget-button"
                onclick="Jupyter.notebook.execute_cells([4]);"> Refresh
                temp </button>
            <button type="button" class="jupyter-button jupyter-widgets mod-warning widget-button"
                onclick="tempToMaster();">Save temp as master</button>
            <button type="button" class="jupyter-button jupyter-widgets mod-primary widget-button"
                onclick="Jupyter.notebook.execute_cells([6]);"> Refresh
                master </button>
            <button type="button" class="jupyter-button jupyter-widgets mod-primary widget-button"
                onclick="document.getElementById('save_conf').style.display = 'block';">Save master to CSV</button>
            <button type="button" class="jupyter-button jupyter-widgets mod-primary widget-button"
                onclick="Jupyter.notebook.execute_cells([8]);"> Update
                Query Output </button>
        </div>
    </div>

    <div id="save_conf" style="display:none">
        <hr>
        <div class="form-group row form-horizontal">
            <h3>Save Configuration</h3>
        </div>
        <div class="form-group row form-horizontal">
            <label class="col-sm-2 control-label">Select Dataset:</label>
            <div class="col-sm-10">
                <div class="widget-dropdown">
                    <select id="datasetSaveSelect" class="form-control">
                    </select>
                </div>
            </div>
        </div>
        <div class="form-group row form-horizontal">
            <label class="col-sm-2 control-label">Result Folder Name:</label>
            <div class="col-sm-10">
                <input id="folderNameSelect" class="form-control"></input>
            </div>
        </div>
        <div class="form-group row form-horizontal">
            <label class="col-sm-2 control-label">Include Header:</label>
            <div class="col-sm-10">
                <input type="checkbox" id="saveHeaderCheckbox" value="with" checked>
                </input>
            </div>
        </div>
        <div class="form-group row form-horizontal">
            <div class="col-sm-offset-2 col-sm-10">
                <button type="button" class="jupyter-button jupyter-widgets mod-primary widget-button"
                    onclick="exportMaster();document.getElementById('save_conf').style.display = 'none';"> Save
                </button>
                <button type="button" class="jupyter-button jupyter-widgets widget-button"
                    onclick="document.getElementById('save_conf').style.display = 'none';"> Cancel </button>
            </div>
        </div>
    </div>

</div>
<script>
    $(document).ready(function () {
        $("#fileBrowser").on('change', function() {
            updateFileBrowserOpenBtn();
        });
        paramDataset = new URL(window.location).searchParams.get("dataset");
        if (paramDataset !== null) {
            paramDataset = "/" + paramDataset;
            QB.currentPath = paramDataset.substring(0, paramDataset.lastIndexOf("/")+1);
        }
        showCurrentBrowser();
        findCurrentProjectID()
            .done(function (projectId) {
                QB.currentProjectId = projectId;
            })
            .done(function () {
                fileBrowserRefresh(paramDataset);
            });
    });
    var QB = {
        currentProjectId: null,
        currentPath: "/",
        browser: {"/":[]}
    }
    var tempDF_columns = []
    var datafile_options = []
    var filter_divs = ["null-filterType", "filterType_joinApproxTime", "filterType_join",
        "filterType_year", "filterType_withColumnRenamed", "filterType_replace",
        "filterType_fillna", "filterType_orderBy", "filterType_count",
        "filterType_min", "filterType_max", "filterType_sum", "filterType_ltrim",
        "filterType_rtrim", "filterType_avg", "filterType_filter", "filterType_filterOr",
        "filterType_select", "filterType_drop", "filterType_isnan",
        "filterType_isnull", "filterType_quarter", "filterType_month",
        "filterType_minute", "filterType_hour", "filterType_dayofmonth", "filterType_dayofyear",
        "filterType_to_date", "filterType_unix_timestamp", "filterType_ytrunc", "filterType_mtrunc",
        "filterType_datediff", "filterType_months_between", "filterType_sqrt", "filterType_round",
        "filterType_when_1", "filterType_when_2", "filterType_when_3", "filterType_col_operation_2",
        "filterType_col_operation_1"]
    var visibleDivId = null;
    var applied_filters = {}
    var filter_count = 0
    var successfulFilters = []
    var successfulFiltersPermanent = [] //< !--temporary solution, to be removed -->

    function show_filters() {
        $("#show-filters").empty()
        $("#show-filters").append("<label class='qblabel'>Selected Filters</label>")
        $.each(applied_filters, function (key, val) {
            if (successfulFiltersPermanent.includes(key)) {
                $("#show-filters").append("<li id='" + key + "' class='applied'><p><em>" + val["descr"] + "</em></p></li>");
            } else {
                $("#show-filters").append("<li id='" + key + "' class='notapplied'><p><em>" + val["descr"] + "</em></p></li>");
            }
        });
    }

    function exportMaster() {
        var kernel = IPython.notebook.kernel;
        var callbacks = {
            iopub: {
                output: csvExported,
            }
        }
        var folder = document.getElementById('folderNameSelect').value;
        var dataset = document.getElementById('datasetSaveSelect').value;
        var withHeader = 0;
        if ($("#saveHeaderCheckbox").is(":checked")) {
            withHeader = 1;
        }
        path = dataset + "/" + folder
        var msg_id = kernel.execute('saveMasterAsCSV("' + path + '",' + withHeader + ')', callbacks);

    }

    // call python to read file from hdfs, reset filter dropdown and populate column dropdowns -->
    function selectDatafile() {
        dataset = document.getElementById('fileSelect').value;
        separator = document.getElementById('sepSelect').value;
        loadDataFile(dataset, separator);
    }

    function loadDataFile(path, separator) {
        var kernel = IPython.notebook.kernel;
        var callbacks = {
            iopub: {
                output: handle_output,
            }
        }
        var msg_id = kernel.execute('read_dataset("' + path + '","' + separator + '")', callbacks);
        divVisibility("null-filterType");
        document.getElementById("filter-application-buttons").style.display = "block"; //because now we have a tempDF
        visibleDivId = null;
        applied_filters = {}
        filter_count = 0
        $("#show-filters").empty()
        successfulFilters = [];
        successfulFiltersPermanent = [];
    }

    function selectDataset() {
        var kernel = IPython.notebook.kernel;
        var callbacks = {
            iopub: {
                output: getCSVfilesFromFS,
            }
        }
        dataset = document.getElementById('datasetSelect').value;
        var msg_id = kernel.execute('walk_dataset("' + dataset + '")', callbacks);
    }

    function getCSVfilesFromFS(data) {
        datafile_options = data.content.text;
        datafilesObtained(JSON.parse(datafile_options.replace(/u\'/g, '"').replace(/\'/g, '"')))
    }

    //add tempDF columns to the dropdown selectors
    function handle_output(data) {
        tempDF_columns = data.content.text;
        addColumnFields(tempDF_columns);
    }

    function filters_applied_result(data) {
        successfulFilters = JSON.parse(data.content.text.replace(/\'/g, '"'))
        for (var i = 0; i < successfulFilters.length; i++) {
            var index = successfulFilters[i];
            applied_filters[index]["executed"] = true;
            successfulFiltersPermanent.push(index);
        }
        show_filters();
        var kernel = IPython.notebook.kernel;
        var callbacks = {
            iopub: {
                output: handle_output,
            }
        }
        var msg_id = kernel.execute('getTempColumns()', callbacks);
    }

    function tempToMaster() {
        var kernel = IPython.notebook.kernel;
        var callbacks = {
            iopub: {
                output: masterDFupdated,
            }
        }
        var msg_id = kernel.execute('tempDFtoMasterDF()', callbacks);
    }

    function csvExported() {
        alert("CSV ready")
    }

    function masterDFupdated(data) {
        masterDF_columns = data.content.text;
        addMasterColumnFields(masterDF_columns);
        alert("The master Dataframe (masterDF) was changed successfully!")
    }

    function populateAvailableDatasetsList(data) {
        $.get({
            url: "/hopsworks-api/api/project/" + data.content.text + "/dataset/getContent", 
            headers: { "Authorization": localStorage.getItem("token") }}, function (data, status) {
            var user_datasets = []

            skipthem = ["Jupyter", "Logs", "Models", "notebook", "Resources"]
            data.forEach(function (element) {
                if (skipthem.includes(element.name) || element.name.endsWith(".db")) {
                    return;
                }
                user_datasets.push(element.path)
            });
            var dropl = document.getElementById("datasetSelect");
            dropl.options.length = 0;
            for (var i = 0; i < user_datasets.length; i++) {
                dropl.options[dropl.options.length] = new Option(user_datasets[i].substring(user_datasets[i].lastIndexOf("/") + 1), user_datasets[i]);
            }
            var droplsave = document.getElementById("datasetSaveSelect");
            droplsave.options.length = 0;
            for (var i = 0; i < user_datasets.length; i++) {
                droplsave.options[droplsave.options.length] = new Option(user_datasets[i].substring(user_datasets[i].lastIndexOf("/") + 1), user_datasets[i]);
            }
        })
    }

    function fileBrowserOpen() {
        var dropl = document.getElementById("fileBrowser");
        if (dropl.options[dropl.selectedIndex].getAttribute("dir") === "true") {
            QB.currentPath = dropl.value;
            fileBrowserRefresh();
        } else {
            console.log("Open file: " + dropl.options[dropl.selectedIndex].getAttribute("path"));
            dataset = dropl.options[dropl.selectedIndex].getAttribute("path");
            separator = document.getElementById('sepSelect').value;
            loadDataFile(dataset, separator);
        }
    }

    function updateFileBrowserOpenBtn() {
        var dropl = document.getElementById("fileBrowser");
        if (dropl.options[dropl.selectedIndex].getAttribute("dir") === "true") {
            $("#fileBrowserOpenBtn").text("Open Folder");
        } else {
            $("#fileBrowserOpenBtn").text("Open File");
        }
    }

    function forceFileBrowserRefresh() {
        var path = QB.currentPath || "/";
        if (path.substr(-1) !== "/") path = path + "/";

        QB.browser[path] = [];
        console.log("force reload files " + path);
        fileBrowserRefresh();
    }

    function fileBrowserRefresh(selectedDataset) {
        var path = QB.currentPath || "/";
        if (path.substr(-1) !== "/") path = path + "/";

        if (!QB.browser[path]) {
            QB.browser[path] = [];
        }

        var b = QB.browser[path];

        if (QB.browser[path].length > 0) {
            console.log("retrieve cached folder: " + path)
            populateFileBrowser(b, selectedDataset);
            return;
        }
        console.log("retrieve folder: " + path)
        $.get({
            url: `/hopsworks-api/api/project/${QB.currentProjectId}/dataset/getContent${path}`, 
            headers: { "Authorization": localStorage.getItem("token") }}, function (data, status) {
            if (path !== "/") {
                var relativePath = path.split("/");
                relativePath.pop();
                relativePath.pop();
                b.push({
                    label: "..",
                    relativePath: relativePath.join("/") + "/",
                    dir: true 
                })
            }

            var shared = path.indexOf("::") > 0;
            skipthem = ["Jupyter", "Logs", "Models", "notebook", "Resources"]
            data.forEach(function (element) {
                if (element.dir && (skipthem.includes(element.name) || element.name.endsWith(".db"))) {
                    return;
                }
                if (!element.dir && element.name.substr(-4) !== ".csv") {
                    return;
                }
                b.push({
                    label: element.name,
                    path: element.path,
                    relativePath: `${path}${element.name}`,
                    dir: element.dir
                });
            });
            b.sort(browserSort);
            populateFileBrowser(b, selectedDataset);
        });
    }
    function populateFileBrowser(b, selectedDataset) {
        var dropl = document.getElementById("fileBrowser");
        dropl.options.length = 0;
        b.forEach(function(bb) {
            dropl.options.add(new Option(bb.label, bb.relativePath));
            dropl.options[dropl.options.length-1].setAttribute("dir", bb.dir);
            if (!bb.dir)
                dropl.options[dropl.options.length-1].setAttribute("path", bb.path);
            if (selectedDataset && selectedDataset === bb.relativePath)
                dropl.options[dropl.options.length-1].selected = true;
        });
        updateFileBrowserOpenBtn();
    }
    // Will sort firt the directories, then the files, sorted by label.
    function browserSort(a, b) {
        if (~a.dir < ~b.dir) {
            return -1;
        }
        if (~a.dir > ~b.dir) {
            return 1;
        }
        if (a.label.toLowerCase() < b.label.toLowerCase()) {
            return -1;
        }
        if (a.label.toLowerCase() > b.label.toLowerCase()) {
            return 1;
        }
        return 0;
    }
    function findCurrentProjectID() {
        var deferred = jQuery.Deferred();
        $.get({
            url: "/hopsworks-api/api/project", 
            headers: { "Authorization": localStorage.getItem("token") }}, function (data, status) {
            var pdict = {}
            data.forEach(function (e) {
                pdict[e.project.name] = e.project.id;
            });
            var kernel = IPython.notebook.kernel;
            var callbacks = {
                iopub: {
                    output: function(data) {
                        var projectId = data.content.text;
                        deferred.resolve(projectId);
                    },
                }
            }
            var pdict_parameter = (JSON.stringify(pdict))
            var msg_id = kernel.execute('find_current_projectID(\'' + pdict_parameter + '\')', callbacks);
        });
        return deferred.promise();
    }

    function showCurrentBrowser() {
        $.get({
            url: "/hopsworks-api/api/project", 
            headers: { "Authorization": localStorage.getItem("token") }}, function (data, status) {
            var pdict = {}
            data.forEach(function (e) {
                pdict[e.project.name] = e.project.id;
            });
            var kernel = IPython.notebook.kernel;
            var callbacks = {
                iopub: {
                    output: populateAvailableDatasetsList,
                }
            }
            var pdict_parameter = (JSON.stringify(pdict))
            var msg_id = kernel.execute('find_current_projectID(\'' + pdict_parameter + '\')', callbacks);
        })
    }

    function datafilesObtained(options) {
        var dropl = document.getElementById("fileSelect");
        dropl.options.length = 0;
        for (opt in options) {
            dropl.options[dropl.options.length] = new Option(options[opt].substring(options[opt].lastIndexOf("/") + 1), options[opt]);
        }
    }

    function divVisibilityChanged() {
        enabled_filter = document.getElementById("filterType").value;
        divid = "filterType_".concat(enabled_filter);
        divVisibility(divid);
    }

    //choose which of the forms will be visible
    function divVisibility(divId) {
        visibleDivId = divId;
        hideNonVisibleDivs();
    }

    //helper:called to choose which of the forms will be visible
    function hideNonVisibleDivs() {
        var i, divId, div;
        for (i = 0; i < filter_divs.length; i++) {
            divId = filter_divs[i];
            div = document.getElementById(divId);
            if (visibleDivId === divId) {
                div.style.display = "block";
            } else {
                div.style.display = "none";
            }
        }
        if ((visibleDivId != null) && (visibleDivId != "null-filterType")) {
            document.getElementById("filter-control-buttons").style.display = "block";
        }
        else {
            document.getElementById("filter-control-buttons").style.display = "none";
        }

    }

    //populate column dropdowns
    function addColumnFields(columns) {
        arraycols = JSON.parse(columns.replace(/\'/g, '"'))
        $('.temp-column').each(function (index) {
            if ($(this).hasClass("empty-option-allowed")) {
                $(this).children('option:not(:first)').remove();
            }
            else {
                $(this).children('option').remove();
            }

            for (var i = 0; i < arraycols.length; i++) {
                optionGenerator($(this), arraycols[i].trim());
            }

        });

    }

    //populate master column dropdowns
    function addMasterColumnFields(columns) {
        masterarraycols = JSON.parse(columns.replace(/\'/g, '"'))
        $('.master-column').each(function (index) {
            $(this).children('option:not(:first)').remove();
            for (var i = 0; i < masterarraycols.length; i++) {
                optionGenerator($(this), masterarraycols[i].trim());
            }

        });
    }

    function applyAllFilters() {
        var kernel = IPython.notebook.kernel;
        var callbacks = {
            iopub: {
                output: filters_applied_result,
            }
        }
        agh = (JSON.stringify(applied_filters))
        lala = {}
        var msg_id = kernel.execute('apply_filters(\'' + agh + '\')', callbacks);
    }

    function cancelFilter() {
        divVisibility('null-filterType');
        $("#filterType")[0].selectedIndex = 0;
    }

    function addFilter() {
        enabled_filter = document.getElementById("filterType").value;
        filterObj = {}
        filter_count++;
        switch (enabled_filter) {
            case "filter": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_filter .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_filter .operator-selector").find(":selected").val())
                parameters.push($("#filterType_filter .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "filterOr": {
                filterObj["type"] = enabled_filter;
                filterObj["executed"] = false;
                parameters = [];
                parameters.push($("#filterType_filterOr .single-column-selector.col1_colname").find(":selected").val());
                parameters.push($("#filterType_filterOr .operator-selector.col1_operator").find(":selected").val());
                parameters.push($("#filterType_filterOr .user-defined-value.col1_value").val());
                parameters.push($("#filterType_filterOr .single-column-selector.col2_colname").find(":selected").val());
                parameters.push($("#filterType_filterOr .operator-selector.col2_operator").find(":selected").val());
                parameters.push($("#filterType_filterOr .user-defined-value.col2_value").val());
                filterObj["parameters"] = parameters;
                filterObj["descr"] = parameters.slice(0,3).toString() + " OR " + parameters.slice(3).toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "orderBy": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push([$("#filterType_orderBy .single-column-selector").find(":selected").val()])
                parameters.push([Number($("#filterType_orderBy .operator-selector").find(":selected").val())])
                filterObj["parameters"] = parameters
                filterObj["descr"] = parameters[0][0] + '(' + $("#filterType_orderBy .operator-selector").find(":selected").text() + ')'
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "count": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push([$("#filterType_count .single-column-selector").find(":selected").val()])
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Count [OrderBy:" + parameters[0][0] + "]"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "ltrim": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_ltrim .single-column-selector").find(":selected").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Left trim [Column:" + parameters[0] + "]"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "rtrim": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_rtrim .single-column-selector").find(":selected").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Right trim [Column:" + parameters[0] + "]"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "isnan": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_isnan .single-column-selector").find(":selected").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Is NaN [Column:" + parameters[0] + "]"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "isnull": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_isnull .single-column-selector").find(":selected").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Is Null [Column:" + parameters[0] + "]"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "select": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                param_array = $("#filterType_select .multiple-column-selector option:selected").map(function () { return this.value }).get();
                parameters.push(param_array);
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Select [" + parameters.toString() + "]"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "drop": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                param_array = $("#filterType_drop .multiple-column-selector option:selected").map(function () { return this.value }).get();
                parameters.push(param_array);
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Drop [" + parameters.toString() + "]"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "min": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push([$("#filterType_min .single-column-selector").find(":selected").val()])
                cols_array = $("#filterType_min .multiple-column-selector option:selected").map(function () { return this.value }).get();
                parameters.push(cols_array);
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Min (" + parameters[1].toString() + ") - Group by " + parameters[0][0]
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "max": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push([$("#filterType_max .single-column-selector").find(":selected").val()])
                cols_array = $("#filterType_max .multiple-column-selector option:selected").map(function () { return this.value }).get();
                parameters.push(cols_array);
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Max (" + parameters[1].toString() + ") - Group by " + parameters[0][0]
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "sum": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push([$("#filterType_sum .single-column-selector").find(":selected").val()])
                cols_array = $("#filterType_sum .multiple-column-selector option:selected").map(function () { return this.value }).get();
                parameters.push(cols_array);
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Sum (" + parameters[1].toString() + ") - Group by " + parameters[0][0]
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "avg": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push([$("#filterType_avg .single-column-selector").find(":selected").val()])
                cols_array = $("#filterType_avg .multiple-column-selector option:selected").map(function () { return this.value }).get();
                parameters.push(cols_array);
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Average (" + parameters[1].toString() + ") - Group by " + parameters[0][0]
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "year": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_year .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_year .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New Year Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "quarter": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_quarter .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_quarter .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New Quarter Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "month": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_month .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_month .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New Month Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "hour": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_hour .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_hour .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New Hour Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "minute": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_minute .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_minute .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New Minute Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "dayofmonth": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_dayofmonth .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_dayofmonth .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New MonthDay Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "dayofyear": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_dayofyear .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_dayofyear .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New YearDay Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "to_date": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_to_date .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_to_date .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New Date Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "unix_timestamp": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_unix_timestamp .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_unix_timestamp .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New UNIX timestamp Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "sqrt": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_sqrt .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_sqrt .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New SQRT Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "round": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_round .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_round .int-input").val().toString())
                parameters.push($("#filterType_round .col-name").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "New round Col:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "ytrunc": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_ytrunc .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_ytrunc .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Year Trunc:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "mtrunc": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_mtrunc .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_mtrunc .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Month Trunc:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "datediff": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_datediff .temp-column-A").find(":selected").val())
                parameters.push($("#filterType_datediff .temp-column-B").find(":selected").val())
                parameters.push($("#filterType_datediff .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "DateDiff:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "months_between": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_months_between .temp-column-A").find(":selected").val())
                parameters.push($("#filterType_months_between .temp-column-B").find(":selected").val())
                parameters.push($("#filterType_months_between .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "MonthDiff:" + parameters.toString();
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "col_operation_1": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_col_operation_1 .temp-column").find(":selected").val())
                parameters.push($("#filterType_col_operation_1 .operator-selector").find(":selected").val())
                parameters.push($("#filterType_col_operation_1 .user-defined-value.operation-value").val())
                parameters.push($("#filterType_col_operation_1 .user-defined-value.new-col-name").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Add column:" + parameters.slice(0, 3).toString() + " (" + parameters[3].toString() + ")";
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "col_operation_2": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_col_operation_2 .temp-column-A").find(":selected").val())
                parameters.push($("#filterType_col_operation_2 .operator-selector").find(":selected").val())
                parameters.push($("#filterType_col_operation_2 .temp-column-B").find(":selected").val())
                parameters.push($("#filterType_col_operation_2 .user-defined-value.new-col-name").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Add column:" + parameters.slice(0, 3).toString() + " (" + parameters[3].toString() + ")";
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "when_1": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_when_1 .temp-column.when-div").find(":selected").val())
                parameters.push($("#filterType_when_1 .operator-selector.when-div").find(":selected").val())
                parameters.push($("#filterType_when_1 .user-defined-value.when-div").val())
                parameters.push($("#filterType_when_1 .user-defined-value.then-div").val())
                parameters.push($("#filterType_when_1 .user-defined-value.else-div").val())
                parameters.push($("#filterType_when_1 .user-defined-value.new-col-name").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "When:" + parameters.slice(0, 3).toString() + " Then:" + parameters[3].toString() + " Else:" + parameters[4].toString() + "(" + parameters[5].toString() + ")";
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "when_2": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_when_2 .temp-column.when-div").find(":selected").val())
                parameters.push($("#filterType_when_2 .operator-selector.when-div").find(":selected").val())
                parameters.push($("#filterType_when_2 .user-defined-value.when-div").val())
                parameters.push($("#filterType_when_2 .temp-column.then-div").find(":selected").val())
                parameters.push($("#filterType_when_2 .operator-selector.then-div").find(":selected").val())
                parameters.push($("#filterType_when_2 .user-defined-value.then-div").val())
                parameters.push($("#filterType_when_2 .temp-column.else-div").find(":selected").val())
                parameters.push($("#filterType_when_2 .operator-selector.else-div").find(":selected").val())
                parameters.push($("#filterType_when_2 .user-defined-value.else-div").val())
                parameters.push($("#filterType_when_2 .user-defined-value.new-col-name").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "When:" + parameters.slice(0, 3).toString() + " Then:" + parameters.slice(3, 6).toString() + " Else:" + parameters.slice(6, 9).toString() + "(" + parameters[9].toString() + ")";
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "when_3": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_when_3 .temp-column.when-div").find(":selected").val())
                parameters.push($("#filterType_when_3 .operator-selector.when-div").find(":selected").val())
                parameters.push($("#filterType_when_3 .user-defined-value.when-div").val())
                parameters.push($("#filterType_when_3 .temp-column-A.then-div").find(":selected").val())
                parameters.push($("#filterType_when_3 .operator-selector.then-div").find(":selected").val())
                parameters.push($("#filterType_when_3 .temp-column-B.then-div").find(":selected").val())
                parameters.push($("#filterType_when_3 .temp-column-A.else-div").find(":selected").val())
                parameters.push($("#filterType_when_3 .operator-selector.else-div").find(":selected").val())
                parameters.push($("#filterType_when_3 .temp-column-B.else-div").find(":selected").val())
                parameters.push($("#filterType_when_3 .user-defined-value.new-col-name").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "When:" + parameters.slice(0, 3).toString() + " Then:" + parameters.slice(3, 6).toString() + " Else:" + parameters.slice(6, 9).toString() + "(" + parameters[9].toString() + ")";
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "fillna": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                pdict = {}
                col = $("#filterType_fillna .single-column-selector").find(":selected").val();
                newv = $("#filterType_fillna .user-defined-value").val();
                pdict[col] = newv
                parameters.push(pdict)
                filterObj["parameters"] = parameters
                filterObj["descr"] = col + ":Fill n/a with " + newv;
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "replace": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push([$("#filterType_replace .user-defined-value.existing-value").val()])
                parameters.push([$("#filterType_replace .user-defined-value.to-be-value").val()])
                parameters.push($("#filterType_replace .single-column-selector").find(":selected").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = parameters[2] + " [" + parameters[0][0] + " --> " + parameters[1][0] + "]"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "withColumnRenamed": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                parameters.push($("#filterType_withColumnRenamed .single-column-selector").find(":selected").val())
                parameters.push($("#filterType_withColumnRenamed .user-defined-value").val())
                filterObj["parameters"] = parameters
                filterObj["descr"] = "Rename " + parameters[0] + " to " + parameters[1]
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "join": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                operator = $("#filterType_join .operator-selector").find(":selected").val()
                tempCol1 = $("#filterType_join .single-column-selector.temp-col-1").find(":selected").val()
                tempCol2 = $("#filterType_join .single-column-selector.temp-col-2").find(":selected").val() || null
                tempCol3 = $("#filterType_join .single-column-selector.temp-col-3").find(":selected").val() || null
                masterCol1 = $("#filterType_join .single-column-selector.master-col-1").find(":selected").val()
                masterCol2 = $("#filterType_join .single-column-selector.master-col-2").find(":selected").val() || null
                masterCol3 = $("#filterType_join .single-column-selector.master-col-3").find(":selected").val() || null
                parameters.push(operator)
                parameters.push([masterCol1, tempCol1])
                parameters.push([masterCol2, tempCol2])
                parameters.push([masterCol3, tempCol3])
                filterObj["parameters"] = parameters
                filterObj["descr"] = operator + "join: (" + masterCol1 + "," + tempCol1 + ")" + "(" + masterCol2 + "," + tempCol2 + ")" + "(" + masterCol3 + "," + tempCol3 + ")"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
            case "joinApproxTime": {
                filterObj["type"] = enabled_filter
                filterObj["executed"] = false
                parameters = []
                operator = $("#filterType_joinApproxTime .operator-selector").find(":selected").val()
                tempTimeCol = $("#filterType_joinApproxTime .single-column-selector.temp-time-col").find(":selected").val()
                masterTimeCol = $("#filterType_joinApproxTime .single-column-selector.master-time-col").find(":selected").val()
                tempCol1 = $("#filterType_joinApproxTime .single-column-selector.temp-col-1").find(":selected").val()
                tempCol2 = $("#filterType_joinApproxTime .single-column-selector.temp-col-2").find(":selected").val() || null
                tempCol3 = $("#filterType_joinApproxTime .single-column-selector.temp-col-3").find(":selected").val() || null
                masterCol1 = $("#filterType_joinApproxTime .single-column-selector.master-col-1").find(":selected").val()
                masterCol2 = $("#filterType_joinApproxTime .single-column-selector.master-col-2").find(":selected").val() || null
                masterCol3 = $("#filterType_joinApproxTime .single-column-selector.master-col-3").find(":selected").val() || null
                parameters.push(operator)
                parameters.push([masterCol1, tempCol1])
                parameters.push([masterCol2, tempCol2])
                parameters.push([masterCol3, tempCol3])
                parameters.push([masterTimeCol, tempTimeCol])
                filterObj["parameters"] = parameters
                filterObj["descr"] = operator + "approx time join on [" + masterTimeCol + "," + tempTimeCol + "]: (" + masterCol1 + "," + tempCol1 + ")" + "(" + masterCol2 + "," + tempCol2 + ")" + "(" + masterCol3 + "," + tempCol3 + ")"
                applied_filters[filter_count.toString()] = filterObj;
                break;
            }
        }
        show_filters();
        divVisibility('null-filterType');
        $("#filterType")[0].selectedIndex = 0;

    }

    function resetAllFilters() {
        applied_filters = []
        filter_count = 0
    }

    //helper: adds options to help populate column dropdowns
    function optionGenerator(instance, paramObj) {
        instance
            .append($("<option></option>")
                .attr("value", paramObj)
                .text(paramObj));
    };
</script>


### Temp Dataset Preview (first 40 rows)


In [ ]:
#P4

if tempDF is not None:
    tempDF.show(40, False)
else:
    print("------------- empty ------------------")


### Master Dataset Preview (first  20 rows)


In [ ]:
#P6

if masterDF is not None:
    masterDF.show(20, False)
else:
    print("------------- empty ------------------")


### Query Output


In [ ]:
#P8
print(query)
print("\n***The code shown here corresponds to the processes that have been applied on the current temporary Dataframe (tempDF)")
